In [ ]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install geneticalgorithm
!pip install mat4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for func-timeout: filename=func_timeout-4.3.5-py3-none-any.whl size=15077 sha256=1913539590001a87d78dd3ba35b0f802ae8a59728070eb9d40f9222fa75945e7
  Stored in directory: /root/.cache/pip/wheels/3f/83/19/b5552bb9630e353f7c5b15be44bf10900afe1abbbfcf536afd
Successfully built func-timeout


In [ ]:
root_path = 'drive/My Drive/TU/Master/MEP/Paper/Working_file_copied_from_final_code/Experiment_3+2b'
import sys, os
#path_dir = os.getcwd()
sys.path.append(root_path+"/Functions")

from general_functions import *
from muscle_functions import *
from integrator_functions import *
from matrices_functions import * #can make specific later if bloated
from constants_functions import *
from plot_functions import *
from save_functions import *

In [ ]:
#pick a trajectory number from Flash experiments, recall optimal trajectory data and store in optimal_trajectory vector
traj_nr = 3
testref_traj1 = mat4py.loadmat('drive/My Drive/TU/Master/MEP/Paper/Working_file_copied_from_final_code/Experiment_3+2b/Optimal_trajectories/ref_traj3.mat')
optimal_trajectories = np.array(testref_traj1['ref_traj'])

In [ ]:

feedback_vector_list = []
cost_list = []
count = 0


lowest_error = 100
lowest_error_vector = []
#feedback_vector =[0.03,0.01,0.00,0.001]

def feedback_function(feedback_vector):

    #The stiffness region that the muscles start in must be in the experiment name. Names are:
    #mostpos,verypos,newpos,newneg,mostneg for stiffness region 1-5. Region name must be preceded by stiff_start_
    experiment_name = '3av25_feedback_stiff_start_verypos_'

    #protocol indicates the system configuration. 'straight_up_delay' for experiment 2 perturbation test, 'reaching' for experiment 3 robustness test
    protocol = 'reaching'#'stroeve','straight_up','straight_up_delay','reaching'

    save = False#If true, the experiment data will be saved to a dataset in /Results/data
    Fmax_muscles = False  #keep False, this is set manually in the constants_functions function

    #model type determines whether Huxley or Hill functions are used, v_type determines the used vce equation in the Huxley model (old is used) and FL_.. determines the used FL model (hill is used)
    model_type,v_type,FL_type='huxley','v_old','FL_hill' #'huxley','hill'---'v_old','v_new'---'FL_0','FL_hill','FL_huxley'
    hill_type = 'stroeve'#determines the hill model used. 'thijs' for hill_expanded, 'stroeve' for hill_simple
    f_type = 'f_damped_0grav'#determines how the force vector in matrices_functions.py is calculated. 'f_damped_fullgrav','f_damped__pgrav','f_damped__0grav','f_undamped_fullgrav','f_undamped__pgrav','f_undamped__0grav' ------------apply manually to matrices_functions.py
    straight_og=False #If True, the current_muscle_starting_lengths will be defined with the arm in the straight up position
    add_feedback = True #If True, the feedback system will be used to change the STIM parameter



    #some experiment parameters
    m3_value = 0.00000001       #allows for manual scaling of ball mass and inertia
    f_factor = 1.0              #alllows for manual scaling of muscle force
    t0 = 0.0                    #time start point (s)
    tf = 0.35                   #time end point   (s)
    t_stimulation = 0.01        #s at this time, the muscle stimulation will start
    t_pertubation = False       #s at this time, the system perturbation will be applied. If False, no perturbation
    h_step = 0.001              #stepsize         (s) #can not change, optimal lengths have the same size
    muscle_number = 6           #nr of muscles used
    tot_bridge_nr = 5000        #number of cross-bridges in the Huxley Model
    step_nr = int((tf-t0)/h_step)
    mus_vals = muscle_vals_func()


    stim_multiplier = 0.25#stim_multiplier_list[multiplier_nr]



    if model_type == 'huxley':
        specific_save_name = str(model_type)+'_stim_mult'+str(feedback_vector[:3])
    if model_type == 'hill':
        specific_save_name = str(model_type)+'_'+str(hill_type)+'_stim_mult'+str(feedback_vector[:3])
    if protocol == 'straight_up':
        starting_a1 = 0.5*sm.pi
    if protocol == 'straight_up_delay':
        starting_a1 = 0.5*sm.pi
    if protocol == 'stroeve':
        starting_a1 = 0.2*sm.pi
    if protocol == 'reaching':
        starting_a1 = 0.5*sm.pi


    l1, l2,l3,l4,m1, m2, J1, J2,F_mtc1,F_mtc2,F_mtc3,F_mtc4,F_mtc5,F_mtc6,r1,r2,B1,B2,thetar1,thetar2,PE_sh1,PE_sh2,PE_xm1,PE_xm2,T_max1,T_max2 = sm.symbols('l_1,l_2,l_3,l_4, m_1, m_2, J_1, J_2, F_{mtc_1},F_{mtc_2},F_{mtc_3},F_{mtc_4},F_{mtc_5},F_{mtc_6},r_1,r_2,B_1,B_2,theta_{r1},theta_{r2},PE_{sh1},PE_{sh2},PE_{xm1},PE_{xm2},T_{max1},T_{max2}')
    t = sm.symbols('t')
    alpha1 = sm.Function('alpha1')(t)
    alpha2 = sm.Function('alpha2')(t)
    alpha3 = sm.Function('alpha3')(t)
    J3, m3 = sm.symbols('J_3,m_3')

    #The joint locations and body center of masses are defined
    x0=0
    y0=0
    x1=0.5*l1*(sm.cos(alpha1))
    y1=0.5*l1*(sm.sin(alpha1))
    x2=2*x1
    y2=2*y1
    x3=x2+0.5*l2*(sm.cos(alpha2))#alpha1 +
    y3=y2+0.5*l2*(sm.sin(alpha2))#alpha1 +
    x4=x2+l2*(sm.cos(alpha2))#alpha1 +
    y4=y2+l2*(sm.sin(alpha2))#alpha1 +

    #Here the starting and ending locations of the muscles are defined, which are later included in the state vector
    #this makes the starts and endings of the muscles scale directly with the joint angle
    #start is defined as the lowest point in the original stroeve image
    xm1_start=-sm.sin(starting_a1)*r1 #-sm.sin(alpha1)*r1
    ym1_start=sm.cos(starting_a1)*r1  #sm.cos(alpha1)*r1
    xm1_end  =x2*l3/l1
    ym1_end  =y2*l3/l1
    xm2_start=sm.sin(starting_a1)*r1#sm.sin(alpha1)*r1
    ym2_start=-sm.cos(starting_a1)*r1#-sm.cos(alpha1)*r1
    xm2_end  =x2*l3/l1
    ym2_end  =y2*l3/l1
    xm3_start=x2*(l1-l4)/l1
    ym3_start=y2*(l1-l4)/l1
    xm3_end  =x2-sm.sin(alpha2)*r1#alpha1 +
    ym3_end  =y2+sm.cos(alpha2)*r1#alpha1 +
    xm4_start=x2*(l1-l4)/l1
    ym4_start=y2*(l1-l4)/l1
    xm4_end  =x2+sm.sin(alpha2)*r1#alpha1 +
    ym4_end  =y2-sm.cos(alpha2)*r1#alpha1 +
    xm5_start=-sm.sin(starting_a1)*r1
    ym5_start=sm.cos(starting_a1)*r1
    xm5_end  =xm3_end
    ym5_end  =ym3_end
    xm6_start=sm.sin(starting_a1)*r1
    ym6_start=-sm.cos(starting_a1)*r1
    xm6_end  =xm4_end
    ym6_end  =ym4_end

    #I might need to redefine these to simply increase or decrease the length in a pulley-like fashion
    l_mtc1=sm.sqrt((xm1_end-xm1_start)**2+(ym1_end-ym1_start)**2)
    l_mtc2=sm.sqrt((xm2_end-xm2_start)**2+(ym2_end-ym2_start)**2)
    l_mtc3=sm.sqrt((xm3_end-xm3_start)**2+(ym3_end-ym3_start)**2)
    l_mtc4=sm.sqrt((xm4_end-xm4_start)**2+(ym4_end-ym4_start)**2)
    l_mtc5=sm.sqrt((xm5_end-xm5_start)**2+(ym5_end-ym5_start)**2)
    l_mtc6=sm.sqrt((xm6_end-xm6_start)**2+(ym6_end-ym6_start)**2)

    x = sm.Matrix([alpha1, x1, y1,alpha2,x3,y3,alpha3,x4,y4])
    q = sm.Matrix([alpha1,alpha2,alpha3])
    T_mat = x.diff(t).jacobian(q.diff(t))

    #define constants and specifieds
    p_protocol_stroeve = {
        l1: 0.32,    # m
        l2: 0.32,  # m
        l3: 0.125,  # m
        l4: 0.125,    #m
        m1: 1.8,    # kg
        J1: 0.015,    # kg*m^2 https://aip-scitation-org.tudelft.idm.oclc.org/doi/pdf/10.1063/1.3479844#:~:text=We%20can%20see%20that%20the,0.1kgm2%20to%200.5kgm2.
        m2: 1.6,    # m
        J2: 0.013,    # kg*m^2
        r1: 0.03,   #rad
        r2: 0.03,   #rad
        m3: 0.00000001,
        J3: 0.0000001, #ball: 2/5*M*r2. Say r of 0.05m, m of 2: 0.002
        B1: 0.3,#Shoulder damping, Nms/rad
        B2: 0.2,#Elbow damping, Nms/rad
        thetar1: 0.2*np.pi,#Shoulder rest angle, rad np.pi/3 OG
        thetar2: 0.2*np.pi,#Elbow rest angle, rad np.pi/2 OG
        PE_sh1: 18,#Shoulder PE shape
        PE_sh2: 18,#Elbow PE shape
        PE_xm1: 0.5*np.pi,#Shoulder PE range, rad
        PE_xm2: 0.5*np.pi,#Elbow PE range, rad
        T_max1: 75,#maximum shoulder torque, winters and stark 85
        T_max2: 51,#maximum elbow torque, winters and stark 85
    }

    p_protocol_straight_up = {
        l1: 0.32,    # m
        l2: 0.32,  # m
        l3: 0.125,  # m
        l4: 0.125,    #m
        m1: 1.8,    # kg
        J1: 0.015,    # kg*m^2 https://aip-scitation-org.tudelft.idm.oclc.org/doi/pdf/10.1063/1.3479844#:~:text=We%20can%20see%20that%20the,0.1kgm2%20to%200.5kgm2.
        m2: 1.6,    # m
        J2: 0.013,    # kg*m^2
        r1: 0.03,   #rad
        r2: 0.03,   #rad
        m3: m3_value,#0.00000001,#
        J3: 0.0008*m3_value, #ball: 2/5*M*r^2. Say r of 0.04m, m of 1: 0.00064. Circle: 0.5*m*r^2v #0.00000001,
        B1: 0.3,#Shoulder damping, Nms/rad
        B2: 0.2,#Elbow damping, Nms/rad
        thetar1: 1/3*np.pi,#Shoulder rest angle, rad
        thetar2: 0.5*np.pi,#Elbow rest angle, rad
        PE_sh1: 18,#Shoulder PE shape
        PE_sh2: 18,#Elbow PE shape
        PE_xm1: 0.5*np.pi,#Shoulder PE range, rad
        PE_xm2: 0.5*np.pi,#Elbow PE range, rad
        T_max1: 75,#maximum shoulder torque, winters and stark 85
        T_max2: 51,#maximum elbow torque, winters and stark 85
    }
    #initial conditions
    init_protocol_stroeve = {
        alpha1: 0.2*np.pi,
        alpha2: 0.2*np.pi,
        alpha3: 0.0*np.pi,
        alpha1.diff(t): 0,
        alpha2.diff(t): 0,
        alpha3.diff(t): 0,
    }
    init_protocol_straight_up = {
        alpha1: 0.500000*np.pi,
        alpha2: 0.50000*np.pi,
        alpha3: 0.0*np.pi,
        alpha1.diff(t): 0,
        alpha2.diff(t): 0.001*np.pi,
        alpha3.diff(t): 0,
    }
    init_protocol_straight_up_delay = {
        alpha1: 0.50*np.pi,
        alpha2: 0.50*np.pi,
        alpha3: 0.0*np.pi,
        alpha1.diff(t): 0.0*np.pi,
        alpha2.diff(t): 0.0*np.pi,
        alpha3.diff(t): 0.0*np.pi,
    }
    init_protocol_reaching = {
        alpha1: 0.2086*np.pi,
        alpha2: 0.7914*np.pi,
        alpha3: 0.0000000001*np.pi,
        alpha1.diff(t): 0.0*np.pi,
        alpha2.diff(t): 0.0*np.pi,
        alpha3.diff(t): 0.0*np.pi,
    }
    pert_protocol_straight_up_delay = {
        alpha1: 0.0*np.pi,
        alpha2: 0.0*np.pi,
        alpha3: 0.0*np.pi,
        alpha1.diff(t): 0.0*np.pi,
        alpha2.diff(t): 0.005*np.pi,
        alpha3.diff(t): 0.0*np.pi,
    }

    if protocol == 'straight_up':
        init = init_protocol_straight_up
        p = p_protocol_straight_up
        t_pertubation = False
        pert_vals = False
    if protocol == 'stroeve':
        init = init_protocol_stroeve
        p = p_protocol_stroeve
        pert_vals = False
    if protocol == 'straight_up_delay':
        init = init_protocol_straight_up_delay
        p = p_protocol_straight_up
        pert_vals = np.array(list(pert_protocol_straight_up_delay.values()))
    if protocol == 'reaching':
        init = init_protocol_reaching
        p = p_protocol_straight_up
        t_pertubation = False
        pert_vals = False

    r = {
        F_mtc1:   0.000000001,  # N
        F_mtc2:   0.000000001,  # N
        F_mtc3:   0.000000001,  # N
        F_mtc4:   0.000000001,  # N
        F_mtc5:   0.000000001,  # N
        F_mtc6:   0.000000001,  # N
    }

    #invoke system calculation functions
    F_newton, M_newton, q_args, qd_args, p_args, r_args = F_M_vars_calc(l1, l2,l3,l4,m1, m2, J1, J2,F_mtc1,F_mtc2,F_mtc3,F_mtc4,F_mtc5,F_mtc6,r1,r2,t,alpha1,alpha2,x0,x1,x2,x3,x4,y0,y1,y2,y3,y4,p,r,J3,m3,alpha3,B1,B2,thetar1,thetar2,PE_sh1,PE_sh2,PE_xm1,PE_xm2,T_max1,T_max2,f_type);
    M_tmt, Q_tmt, C_M, C_Q, A, b, eval_A, eval_b, eval_C, eval_Cd, eval_Cq, g_conv = matrices_func(M_newton, F_newton, q_args, qd_args, p_args, r_args, T_mat,x,q,t);
    init_vals = np.array(list(init.values()))

    #Make arrays of the values in order for use in the numerical functions
    q_vals = np.array([init_vals[0],init_vals[1],init_vals[2]])
    qd_vals = np.array([init_vals[3], init_vals[4], init_vals[5]])
    p_vals = np.array([p[p_sym] for p_sym in p_args])
    r_vals = np.array([r[r_sym] for r_sym in r_args])
    eval_xs = sm.lambdify((q_args, p_args), [x0,x0,x1,x2,x3,x4])
    eval_ys = sm.lambdify((q_args, p_args), [y0,y0,y1,y2,y3,y4])
    eval_ls = sm.lambdify((q_args, p_args), [l_mtc1, l_mtc2,l_mtc3,l_mtc4,l_mtc5,l_mtc6])

    #make an input stimulation vector
    step_nr = int((tf-t0)/h_step)
    input_STIM = [[]]*muscle_number
    starting_step = int((t_stimulation/tf)*step_nr)
    input_STIM[0] = np.ones(step_nr+2)*stim_multiplier*1
    input_STIM[1] = np.ones(step_nr+2)*stim_multiplier*1
    input_STIM[2] = np.ones(step_nr+2)*stim_multiplier*1
    input_STIM[3] = np.ones(step_nr+2)*stim_multiplier*1
    input_STIM[4] = np.ones(step_nr+2)*stim_multiplier*1
    input_STIM[5] = np.ones(step_nr+2)*stim_multiplier*1
    input_STIM[0][0:starting_step] = 0 #this way, step input should be applied at 0.01 s
    input_STIM[1][0:starting_step] = 0
    input_STIM[2][0:starting_step] = 0
    input_STIM[3][0:starting_step] = 0
    input_STIM[4][0:starting_step] = 0
    input_STIM[5][0:starting_step] = 0

    #create function to iterate over with rk4
    eval_rhs2 = lambda t, s, r_vals: eval_rhs(t, s, p_vals, r_vals,eval_A, eval_b)[0]  # don't output the Lagrange multipliers

    for kkk in range(muscle_number):#scale the maximum force variables
        mus_vals[kkk][7]*=f_factor

    #execute calculations
    t_vals, s_vals, coords_vals,mtc_vels,calculated_muscle_forces,states,activation,mus_vals_changed,K_SE_list,K_CE_list,force_vector = rk4_integrator(eval_rhs2, t0, tf, init_vals, t_pertubation,pert_vals,h_step, r_vals, muscle_number, tot_bridge_nr, eval_xs, eval_ys, eval_ls,p_vals,mus_vals,Fmax_muscles,f_factor,False,True,input_STIM,True,False,model_type,hill_type,v_type,FL_type,straight_og,specific_save_name,experiment_name,optimal_trajectories,feedback_vector,add_feedback)

    #some data preparation
    l_mtcs = [[]]*muscle_number
    for muscle in range(muscle_number):
        l_mtcs[muscle] = coords_vals[:,2][:,muscle]
    if model_type =='huxley':
        lCEs = np.zeros((muscle_number,len(states[0][:,-2])))
        lSEs = np.zeros((muscle_number,len(states[0][:,-2])))
        for i in range(muscle_number):
            lCEs[i] = states[i][:,-2]
        for i in range(muscle_number):
            lSEs[i] = coords_vals[:,2,i] - lCEs[i]
    if model_type =='hill':
        if hill_type == 'stroeve':
            lCEs = np.zeros((muscle_number,len(states[0])))
            vCEs = np.zeros((muscle_number,len(states[0])))
            for muscle in range(muscle_number):
                lCEs[muscle] = l_mtcs[muscle] - 0.02
                vCEs[muscle] = np.gradient(lCEs[muscle],h_step)
        if hill_type == 'thijs':
            lCEs = np.zeros((muscle_number,len(states[0])))
            vCEs = np.zeros((muscle_number,len(states[0])))
            for muscle in range(muscle_number):
                lCEs[muscle] = states[muscle][:,0]
                vCEs[muscle] = force_vector[muscle][:,7]
    optimal_muscle_length_array = np.zeros((len(t_vals),muscle_number))
    for a in range(len(t_vals)):
        if a <=750: #length of optimal value vector
            optimal_muscle_length_array[a] = eval_ls([optimal_trajectories[:,1][a-1],optimal_trajectories[:,2][a-1]+optimal_trajectories[:,1][a-1],0],p_vals)#determine optimal muscle lengths at this time
        if a >750: #length of optimal value vector
            optimal_muscle_length_array[a] = eval_ls([optimal_trajectories[:,1][750],optimal_trajectories[:,2][750]+optimal_trajectories[:,1][750],0],p_vals)#determine optimal muscle lengths at this time

                #save data
    if save == True:
        if model_type =='huxley':
            save_huxley(l_mtcs,tot_bridge_nr,states,force_vector,mtc_vels,input_STIM,lCEs,lSEs,K_CE_list,K_SE_list,coords_vals,s_vals,t_vals,experiment_name, specific_save_name)
        if model_type =='hill':
            save_hill(l_mtcs,calculated_muscle_forces,force_vector,mtc_vels,input_STIM,lCEs,vCEs,t_vals,s_vals,experiment_name,specific_save_name)


    # if model_type =='huxley':
    #     plot_huxley_single_muscle(t_vals, s_vals, coords_vals,mtc_vels,calculated_muscle_forces,states,input_STIM,t0,tf,t_pertubation,h_step,muscle_number,tot_bridge_nr,step_nr,mus_vals_changed,np.array(list(p.values())),v_type,FL_type,f_type,K_SE_list,K_CE_list,lCEs,lSEs,save,experiment_name,specific_save_name)

    #plot_movement_static(t_vals, s_vals, coords_vals,mtc_vels,calculated_muscle_forces,states,input_STIM,t0,tf,h_step,muscle_number,tot_bridge_nr,step_nr,mus_vals,save,experiment_name,specific_save_name)

    opt_a1 = np.ones(len(t_vals))*optimal_trajectories[-1,1]
    opt_a2 = np.ones(len(t_vals))*(optimal_trajectories[-1,2]+optimal_trajectories[-1,1])
    opt_a3 = np.ones(len(t_vals))*optimal_trajectories[-1,3]
    opt_a4 = np.ones(len(t_vals))*(optimal_trajectories[-1,4]+optimal_trajectories[-1,3])
    if int(tf/h_step) > len(optimal_trajectories[:,1]):
        opt_a1[:len(optimal_trajectories[:,1])] = optimal_trajectories[:,1]
        opt_a2[:len(optimal_trajectories[:,2])] = optimal_trajectories[:,2]+optimal_trajectories[:,1]
        opt_a3[:len(optimal_trajectories[:,3])] = optimal_trajectories[:,3]
        opt_a4[:len(optimal_trajectories[:,4])] = optimal_trajectories[:,4]+optimal_trajectories[:,3]
    if int(tf/h_step) <= len(optimal_trajectories[:,1]):
        opt_a1[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),1]
        opt_a2[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),2]+optimal_trajectories[:int(tf/h_step),1]
        opt_a3[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),3]
        opt_a4[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),4]+optimal_trajectories[:int(tf/h_step),3]

    opt_x=[[]]*len(opt_a1)
    opt_y=[[]]*len(opt_a1)
    opt_x = eval_xs([opt_a1,opt_a2,0],p_vals)#determine optimal muscle lengths at this time
    opt_y = eval_ys([opt_a1,opt_a2,0],p_vals)
    #print(opt_x)
    #integrated_trajectory_error = np.trapz((s_vals[:,1]-opt_a2)**2)+2*np.trapz((s_vals[:,0]-opt_a1)**2)
    #integrated_trajectory_error = np.trapz((np.sqrt(opt_x[-1]**2+opt_y[-1]**2)-np.sqrt(coords_vals[:,1,-1]**2+coords_vals[:,0,-1]**2))**2)#*10+np.trapz((s_vals[:,0]-opt_a1)**2)#+np.trapz((s_vals[:,3]-opt_a3)**2)+np.trapz((s_vals[:,4]-opt_a4)**2)
    integrated_trajectory_error = np.trapz(np.sqrt((opt_x[-1]-coords_vals[:,0,-1])**2+(opt_y[-1]-coords_vals[:,1,-1])**2))
    global lowest_error
    global lowest_error_vector
    if integrated_trajectory_error < lowest_error:
      lowest_error = integrated_trajectory_error
      print('New lowest error: '+str(integrated_trajectory_error))
      lowest_error_vector = feedback_vector
      print('Associated feedback vector: '+str(feedback_vector))

    #if integrated_trajectory_error <= 0.01:
        #plot data
        #if model_type =='huxley':
        #    plot_huxley_full(t_vals, s_vals, coords_vals,mtc_vels,calculated_muscle_forces,states,input_STIM,t0,tf,t_pertubation,h_step,muscle_number,tot_bridge_nr,step_nr,mus_vals_changed,np.array(list(p.values())),v_type,FL_type,f_type,K_SE_list,K_CE_list,lCEs,lSEs,save,experiment_name,specific_save_name,optimal_trajectories,optimal_muscle_length_array)
        #if model_type == 'hill':
        #    plot_hill_full(t_vals, s_vals, coords_vals,mtc_vels,calculated_muscle_forces,states,input_STIM,t0,tf,t_pertubation,h_step,muscle_number,tot_bridge_nr,step_nr,mus_vals,save,experiment_name,specific_save_name,optimal_trajectories,optimal_muscle_length_array)
    global feedback_vector_list
    global cost_list
    global count

    #feedback_vector_list[count] = feedback_vector
    #cost_list[count] = integrated_trajectory_error
    feedback_vector_list.append(feedback_vector)
    cost_list.append(integrated_trajectory_error)
    count += 1
    return integrated_trajectory_error


In [ ]:
#run the genetic algorithm to find optimal feedback vector
varbound=np.array([[-50,100],[-50,50],[-20,20],[-20,20]])
algorithm_param = {'max_num_iteration': 600,\
                   'population_size':100,\
                   'mutation_probability':0.3,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':30}
model=ga(function=feedback_function,dimension=4,variable_type='real',variable_boundaries=varbound,algorithm_parameters=algorithm_param)

model.run()


New lowest error: 10.892979602532083
Associated feedback vector: [-31.11656532   0.83091059   3.20941071  -8.93784478]
New lowest error: 5.161848703854899
Associated feedback vector: [ 97.14533496 -38.45823857 -15.28508603 -12.27950014]
New lowest error: 4.105375806362018
Associated feedback vector: [ -6.28429267 -15.50765196  10.28611237 -10.62962587]
__________________________________________________ 0.7% GA is running...New lowest error: 4.096223698082206
Associated feedback vector: [ -4.47182004 -29.85848223  10.28611237 -10.62962587]
__________________________________________________ 1.0% GA is running...New lowest error: 4.09536462448183
Associated feedback vector: [ 45.99732776 -31.66176216  10.28611237 -10.62962587]
|_________________________________________________ 1.8% GA is running...New lowest error: 4.079221251586583
Associated feedback vector: [ 32.15803554 -47.93409352  10.28611237 -10.62962587]
||________________________________________________ 3.5% GA is running...New 

KeyboardInterrupt: 

In [ ]:
# 29/09/2024
'''
4.09536462448183 [ 45.99732776 -31.66176216  10.28611237 -10.62962587]
4.079221251586583
Associated feedback vector: [ 32.15803554 -47.93409352  10.28611237 -10.62962587]
New lowest error: 4.07858514867002
Associated feedback vector: [  3.672605   -47.93409352  10.28611237 -10.62962587]
New lowest error: 4.0774720360955214
Associated feedback vector: [-44.20130991 -47.93409352  10.28611237 -10.62962587]


'''






#earlier optimisation attempts
#[146.61424623   0.41363624   1.29514679  -1.39239114] with cost  0.29690510976051593
#[66.29747539 -1.42389007  0.36790602 -0.39515029]  0.008279244494811551
#[-44.46750588  -1.4138054    0.36790602  -0.39515029] 0.009036026177256531
#[ 5.57172165 -1.4921973   0.36790602 -0.39515029] 0.00929917329678755
#

#stroeve
#0.02286973568885877 [90.08570008 -9.76164739 -1.6795049   0.3007247 ]
#New lowest error: 0.022389414267146347
#Associated feedback vector: [74.66824162 -0.92642477  0.7147669  -0.71703921]

cost_list

[3.6453109987505505,
 4.3084493614629595,
 3.6453109987505505,
 3.47859951044387,
 3.939237240223061,
 3.461621954853123,
 4.021547704588674,
 3.8947553381087565,
 4.192805112383869,
 2.5913386949035377,
 4.839145955401305,
 3.510053453232064,
 7.658635598341704,
 3.9482286759716057,
 3.545644514619096,
 3.7344398186446615,
 6.02361069468315,
 4.743689295353597,
 4.268883535905189,
 3.333410106784214,
 7.658635598341704,
 2.3542210579262353,
 4.3163830863701005,
 4.297547216880058,
 3.938935861147643,
 4.324355657730332,
 3.9310517750838407,
 3.6453109987505505,
 2.1523820882609987,
 3.866100974136724,
 4.297780126501953,
 4.201649622634901,
 3.7099547885564794,
 3.9591428912058735,
 3.9202480649741545,
 4.131136095861469,
 4.436172236596603,
 3.7654356462698084,
 3.6727968382750253,
 4.011746902352838,
 2.6212865345137737,
 4.19187185826316,
 1.9818272833008157,
 4.4605755688870925,
 4.141813754321191,
 7.6965933366937715,
 2.0698057292249326,
 7.4923782820729095,
 4.301335515495471,


In [ ]:

#feedback_vector = np.ones(4*6)*0.1

best_costs_list = []
best_fbvector_list = []
total_opt_iterations = 20
for optimiser_trial in range(total_opt_iterations):
    feedback_vector_list = []
    cost_list = []
    count = 0
    feedback_vector0 = np.random.uniform(-1,2,size=4)
    scipy.optimize.minimize(feedback_function, feedback_vector0, method="nelder-mead",tol=0.01,options={'maxiter':100})
    #BFGS
    if optimiser_trial > 0:
        del saved_data
        del min_iter
        del min_fbvector
    saved_data = xr.Dataset(
        data_vars=dict(
            costs = (['iteration'],cost_list),
            feedback_parameters = (['iteration','fbvar_nr'],feedback_vector_list),
        ),
        attrs=dict(description="cost and feedback parameters over time"),
        )

    saved_data.to_netcdf('Results/data/traj'+str(traj_nr)+'_optimdata_'+str(count)+'its_fb0_'+str(feedback_vector0))

    min_iter = saved_data.iteration[abs(saved_data.costs)==abs(saved_data.costs).min()]
    min_fbvector = saved_data.feedback_parameters[saved_data.iteration==min_iter[0]]
    min_cost = saved_data.costs[saved_data.iteration==min_iter[0]]
    print('Lowest found cost in optimiser trial '+str(optimiser_trial)+': '+str(min_cost.values))
    print('Accompanying feedback vector: '+str(min_fbvector.values))
    print('Initial feedback vector: '+str(feedback_vector0))
    best_costs_list.append(min_cost)
    best_fbvector_list.append(min_fbvector)
    #plt.close()
    plt.plot(saved_data.costs[:])
best_data = xr.Dataset(
    data_vars=dict(
        best_costs = (['optimiser_trial'],best_costs_list),
        best_feedback_vectors = (['optimiser_trial','fbvar_nr'],best_fbvector_list),
    ),
    attrs=dict(description="cost and feedback parameters of local optima"),
    )
best_data.to_netcdf('Results/data/traj'+str(traj_nr)+'_best_optimdata_'+time.ctime())


In [ ]:
'''
#found wth 29.03 for angles
1.37815801e-03,  3.52666067e-03,  1.23255705e-06, -1.40891890e-03,
       -2.13122654e-03,  4.38530628e-06, -5.57971545e-03,  7.63760180e-03,
       -4.29388059e-03,  4.66600563e-03,  2.40302068e-03, -1.01337729e-05,
       -3.37229682e-03,  6.42850089e-03, -9.98706055e-04, -3.74069856e-03,
       -2.98774458e-03, -6.39792787e-06
#found with 5.5
[-5.92180551e+00,  6.45928371e+00,  1.01560160e-02, -2.04020821e-01,
       -3.36797198e+01,  2.62443619e-02,  3.50233101e+01,  1.53717879e+01,
        5.56498212e-03, -3.71168174e+01,  1.79729309e+01,  3.67195231e-03,
        1.92485382e+00, -1.70453091e-01,  8.30853636e-02, -1.97827448e+01,
        4.00582014e+01,  1.24554985e-01
[-5.93209550e+00,  6.48686315e+00,  1.01465431e-02, -2.04890752e-01,
       -3.40910348e+01,  2.68195057e-02,  3.51988281e+01,  1.53928540e+01,
        5.58124651e-03, -3.58898523e+01,  1.80308753e+01,  3.68294095e-03,
        1.93536556e+00, -1.72095997e-01,  8.35923000e-02, -1.94314082e+01,
        4.04586135e+01,  1.24554279e-01]

try 2, mostly shit
[-5.92529677e+00,  6.81752722e+00,  1.01779716e-02, -2.10819773e-01,
       -3.42253326e+01,  2.68597800e-02,  3.43435620e+01,  1.54536292e+01,
        5.59388523e-03, -3.61300736e+01,  1.80907349e+01,  3.69290609e-03,
        1.94526809e+00, -1.72607928e-01,  8.36258262e-02, -1.86517982e+01,
        4.19049934e+01,  1.28032385e-01]
hux 47 #v2
[-2.60904972e+01, -4.34188718e+01, -6.78404733e-04, -2.75031703e-01,
       -4.23711095e+01,  1.01698915e-02,  1.75494159e+02,  6.03970724e+01,
        1.84180750e-03,  7.23231937e+01, -5.75343466e+01,  3.37629415e-03,
        1.43473673e+01, -5.16251198e-01,  4.11713462e-03, -9.06670270e+01,
       -1.93382946e+02,  1.90164306e-01]
[-5.55496973e+00,  6.14683613e+00,  1.08274161e-02, -2.09385719e-01,
       -3.50361716e+01,  2.49540995e-02,  3.36152355e+01,  1.60158364e+01,
        5.17503954e-03, -3.53752385e+01,  1.76368636e+01,  4.00914349e-03,
        1.95291778e+00, -1.77546926e-01,  8.88565199e-02, -2.00673894e+01,
        3.93464572e+01,  1.20070437e-01]
hux 0.1 #v2
[-5.62313661e+00,  6.36306988e+00,  1.14256497e-02,
        -2.10157139e-01, -3.44196820e+01,  2.43346475e-02,
         3.44203449e+01,  1.57496772e+01,  5.40515389e-03,
        -3.62247850e+01,  1.81006859e+01,  3.79599869e-03,
         1.91685304e+00, -1.72962144e-01,  8.31940336e-02,
        -2.04440199e+01,  3.93894547e+01,  1.22374066e-01]]
hux 0.01 #v4
[ 1.72008006e-04, -6.85387544e-05, -4.00199304e-06,
        -5.03143528e-05, -5.15540167e-05,  1.67216676e-04,
        -1.26227411e-04, -8.37423955e-05, -5.78024292e-05,
         4.50414946e-05, -1.61406043e-04,  2.85493240e-05,
         2.89115553e-05, -7.37420250e-05,  5.87097876e-04,
         1.73516983e-05, -8.74976493e-05, -1.11522919e-05,
        -2.02513970e-04,  2.25109394e-04, -5.33683607e-05,
        -7.35652865e-05, -2.91904664e-06,  1.23066953e-04]
hux #5 ga cost 0.03885
[ 0.46602414  0.47485954 -0.41222565  0.03347365]

0.013973
[ 1.01770968, -0.71289673, -0.30396472,  0.01905234]

0.012349
[ 1.9679346  -1.01729415 -0.13599131 -0.02291269]


Lowest found cost in optimiser trial 10: [0.01098226]
Accompanying feedback vector: [[ 0.27315625 -0.5619365   0.0568637  -0.14411725]]
Initial feedback vector: [ 0.53594739 -0.46179852  0.32649322 -0.134069  ]

Lowest found cost in optimiser trial 1: [0.04442229]
Accompanying feedback vector: [[-0.92224806 -0.69097505 -0.72491863  0.03496949]]
Initial feedback vector: [-0.99260167 -0.3821575  -0.65306308 -0.64148581]



hill
 [-0.32864914,-1.49792975,-0.0763267,-0.23360793] works but weird





'''

In [ ]:
prevdata =  xr.open_dataset('Results/data/traj3_optimdata_1657its_Wed Feb 22 21:14:34 2023')
min_iter = prevdata.iteration[abs(prevdata.costs)==abs(prevdata.costs).min()]
print(prevdata.feedback_parameters[prevdata.iteration == min_iter[0]])

In [ ]:
opt_a1 = np.ones(len(t_vals))*optimal_trajectories[-1,1]
opt_a1[:len(optimal_trajectories[:,1])] = optimal_trajectories[:,1]
opt_a2 = np.ones(len(t_vals))*(optimal_trajectories[-1,2]+optimal_trajectories[-1,1])
opt_a2[:len(optimal_trajectories[:,2])] = optimal_trajectories[:,2]+optimal_trajectories[:,1]
plt.plot(t_vals,s_vals[:,0],label='a1')
plt.plot(t_vals,s_vals[:,1],label='a2')
plt.plot(t_vals,opt_a1,label='a1 optimal')
plt.plot(t_vals,opt_a2,label='a2 optimal')
plt.xlabel('time [s]')
plt.ylabel('joint angle [rad]')
plt.legend(loc='right')
plt.title('Simulated vs optimal joint angles')
len(coords_vals[1])

In [ ]:
print(min_iter.values)

In [ ]:
opt_a1 = np.ones(len(t_vals))*optimal_trajectories[-1,1]
opt_a2 = np.ones(len(t_vals))*(optimal_trajectories[-1,2]+optimal_trajectories[-1,1])
opt_a3 = np.ones(len(t_vals))*optimal_trajectories[-1,3]
opt_a4 = np.ones(len(t_vals))*(optimal_trajectories[-1,4]+optimal_trajectories[-1,3])
if int(tf/h_step) > len(optimal_trajectories[:,1]):
    opt_a1[:len(optimal_trajectories[:,1])] = optimal_trajectories[:,1]
    opt_a2[:len(optimal_trajectories[:,2])] = optimal_trajectories[:,2]+optimal_trajectories[:,1]
    opt_a3[:len(optimal_trajectories[:,3])] = optimal_trajectories[:,3]
    opt_a4[:len(optimal_trajectories[:,4])] = optimal_trajectories[:,4]+optimal_trajectories[:,3]
if int(tf/h_step) <= len(optimal_trajectories[:,1]):
    opt_a1[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),1]
    opt_a2[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),2]+optimal_trajectories[:int(tf/h_step),1]
    opt_a3[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),3]
    opt_a4[:int(tf/h_step)] = optimal_trajectories[:int(tf/h_step),4]+optimal_trajectories[:int(tf/h_step),3]

opt_x=[[]]*len(opt_a1)
opt_y=[[]]*len(opt_a1)
opt_x = eval_xs([opt_a1,opt_a2,0],p_vals)#determine optimal muscle lengths at this time
opt_y = eval_ys([opt_a1,opt_a2,0],p_vals)
opt_x

In [ ]:
total_opt_iterations = 20
for optimiser_trial in range(total_opt_iterations):
    feedback_vector0 = np.random.uniform(-1,2,size=4)
    print(feedback_vector0)